In [320]:
import os
import numpy as np
import re
from operator import add
import dicom as dc
import matplotlib.pyplot as plt
import matplotlib.patches as patches


In [321]:
def sort_by_number(list_):
    
    def atoi(text):
        return int(text) if text.isdigit() else text

    def natural_keys(text):
        return [ atoi(c) for c in re.split('(\d+)', text) ]
    ret_list=list_.sort(key = natural_keys)
    return ret_list


In [322]:
def get_number_from_list(list_):
    """
    모든 폴더에 있는 파일의 순서를 지정할수 있는 숫자를 얻어낸후 리스트로 리턴한다 
    """
    ret_list=[]
    for i,file_name in enumerate(list_):
        ret_list.append(get_number(file_name))
    ret_list=map(int ,ret_list)
    return ret_list
def get_number(file_name):
    """
    1.3.12.2.1107.5.1.4.51914.30000006121806122618700000702.dcm
    -->30000006121806122618700000702.dcm
    -->name[-3:]=702
    """
    extension=file_name.split('.')[-1]
    name=file_name.split('.')[-2]
    name=name[-3:]
    """
    """
    return name
def make_dict(list_name , list_path):
    """
    key:02 value:1.3.12.2.1107.5.1.4.51914.30000006121806122618700000702.dcm
    """
    ret_dic={}
    for i,ele in enumerate(list_path):
        ret_dic[list_name[i]]=ele
    return ret_dic
def attach_(arg1 , arg2):
    """
    arg1=elsement_wise 로 붙일 것
    arg2=list
    /home/seongjung/ASAN_project1/project1/CCM_Data/Benign/001/roimask_0013.dcm'
    """
    list_=[]
    for i in range(len(arg2)):        
        list_.append(arg1)
    list_add=map(add,list_,arg2)    
    return list_add

def find_mask(mask_dic):
    cancer_mask={}
    for i,ele in enumerate(mask_dic.keys()):

        dc_file=dc.read_file(mask_dic[ele])
        dc_img=dc_file.pixel_array
        np_img=np.asarray(dc_img)
   
        if np.mean(dc_img)>0:
            "마스크에서 종양의 여부를 식별합니다"+str(ele)+"번째에서 종양이 발견되었습니다"
            #print ele
            #print np.mean(np_img)
            cancer_mask[ele]=mask_dic[ele]
    return cancer_mask


In [323]:
def dicom_pixel(path):
    dc_file=dc.read_file(path)
    dc_img=dc_file.pixel_array
    return dc_img


In [324]:
def mask_locate(target_mask):
    locate_mask={}
    print "target_mask.keys():{}".format(target_mask.keys())
    for i,key in enumerate(target_mask.keys()):
        try:
            np_img=dicom_pixel(target_mask[key])
            row,col=np.shape(np_img)
            top_flag=False
            buttom_flag=False
            left_flag=False
            right_flag=False
            for r in range(row):
                if np.mean(np_img[r,:])>0 and top_flag==False:
                    #print r
                    top_line=np_img[r,:]>0
                    for i,ele in enumerate(top_line):
                        if ele==True:
                            top= (r,i)

                            break;

                    top_flag=True
                if top_flag==True and np.mean(np_img[r,:])==0 and buttom_flag==False:        
                    buttom_line=np_img[r-1,:]>0

                    #print r-1
                    for i,ele in enumerate(buttom_line):
                        if ele==True:
                            buttom= (r-1,i)
                            break;
                    buttom_flag=True
           

            left_flag=False
            right_flag=False
            crop_np_img=np_img[top[0]:buttom[0],:]
            for c in range(col):
                if left_flag == False and np.mean(np_img[top[0]:buttom[0],c])>0:
                    left_line = np_img[top[0]:buttom[0],c]>0
                    for i , ele in enumerate(left_line):
                        if ele==True:
                            #print i
                            left = (top[0]+i,c)
                            #print left
                            break;
                    left_flag=True
                if left_flag == True and right_flag==False and np.mean(np_img[top[0]:buttom[0],c])==0:
                    right_line =np_img[top[0]:buttom[0],c-1]>0
                    #print right_line
                    for i , ele in enumerate(right_line):
                        if ele==True:
                            #print i
                            right = (top[0]+i,c-1)
                            #print right
                            break;

                    right_flag =True
            
            center_row = ( buttom[0]-top[0])/2
            center_col = ( right[1]-left[1])/2
            center= (top[0]+center_row) , (left[1]+center_col)
            #print center
            locate_list=[top,buttom,left,right,center]
            locate_mask[key]=locate_list
        except :
            print "error info "
            print "mask : ",key
            print "mask location:",target_mask[key]
            continue;
        print 'top , buttom',top , buttom
        print 'left , right',left , right
    return locate_mask

In [325]:
a=[1,2,3]
b=[1,2,3]
attach_(1,a)

[2, 3, 4]

In [326]:
a=[1,2,3]


In [327]:
def crop_image(img , center, row,col):
    return img[center[0]-(row/2) : center[0]+(row/2) , center[1]-(col/2): center[1]+(col/2)]


In [328]:
def draw_rectangle_and_save(x,y,np_img ,save_path):

    # Create figure and axes
    fig,ax = plt.subplots(1)
    # Display the image
    ax.imshow(np_img)
    plt.savefig(save_path+'.png')
    
    # Create a Rectangle patch
    rect = patches.Rectangle((x,y),50,50,fill=True,linewidth=1,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    
    plt.savefig(save_path+'_rect.png')
    plt.close(plt.gcf())


In [329]:
def crop_mask_target_images_save(mask_dic, mask_locate ,target_dic,save_path):

    print "이미지를 crop 하고 저장합니다 "
    """center 좌표를 이용해 mask 이미지와 target data의 이미지를 자른다
    mask data path 을 가지고 있는 arg: mask_dic
    center좌표를 가지고 있는 arg : mask_locate
    target data path 을 가지고 있는 arg : target_dic
    
    이미지로 저장하냐 
    numpy 로 저장 
    엔지니어 관점 문제점 파일을 2번 열어야 되서 효율성에서 떨어진다 코드를 수정해야 한다.
    넘피로도 저장을 해야 한다 .
    
    target_dic.keys()이가 항상 오름차순이라고 가정한다. 
    확인 하는 코드를 넣어야 한다.
    """
    for i ,key in enumerate(mask_dic.keys()):
        print 'key',key
        #print 'key type' ,type(key)
        key=int(key)
        mask_path=mask_dic[key]
        target_list=sorted(target_dic.keys())
        target_key=target_list[-key]
        target_key_up=target_list[key]
        print 'target_key 시작 위치 :'+str(target_list[0])
        print 'target_key 끝 위치 :'+str(target_list[-1])
        
        print 'target_key:'+str(target_key)
        print 'target_key_up:'+str(target_key_up)
        target_path=target_dic[target_key]
        target_path_up=target_dic[target_key_up]
        print 'target_path',target_path
        mask_img=dicom_pixel(mask_path)
        target_img=dicom_pixel(target_path)
        target_img_up=dicom_pixel(target_path_up)
        
        
        
        center=mask_locate[key][4]
        crop_mask_img=crop_image(mask_img  , center , 100 , 100)
        crop_target_img=crop_image(target_img  , center , 100 , 100)
        crop_target_img_up=crop_image(target_img_up  , center , 100 , 100)
        
        mkdir(save_path,str(target_key))
        
        draw_rec_mask_img=draw_rectangle_and_save(25,25,crop_target_img_up,save_path+'/'+str(target_key)+'/crop_target_up_'+str(target_key_up))
        draw_rec_mask_img=draw_rectangle_and_save(25,25,crop_target_img,save_path+'/'+str(target_key)+'/crop_target_'+str(target_key))
        draw_rec_target_img=draw_rectangle_and_save(25,25,crop_mask_img,save_path+'/'+str(target_key)+'/crop_mask_'+str(key))
        
        draw_rec_target_img=draw_rectangle_and_save(center[1]-25,center[0]-25,target_img_up,save_path+'/'+str(target_key)+'/target_up_'+str(target_key_up))
        draw_rec_target_img=draw_rectangle_and_save(center[1]-25,center[0]-25,target_img,save_path+'/'+str(target_key)+'/target_'+str(target_key))
        draw_rec_target_img=draw_rectangle_and_save(center[1]-25,center[0]-25,mask_img,save_path+'/'+str(target_key)+'/mask_'+str(key))
    
    
        
        
        
        
        

In [330]:
def mkdir(path,name=None , secceed_on=True):
    if os.path.isdir(path)==False:
        os.mkdir(path)
    if os.path.isdir(path)==True:
        if name != None and os.path.isdir(path+'/'+name)==False:
            os.mkdir(path+'/'+name)

In [331]:
folder_path='/home/seongjung/ASAN_project1/project1/CCM_Data/Benign/'
save_foler='/home/seongjung/ASAN_DATA/Benign'
folder_list = os.walk(folder_path).next()[1]
folder_list=sorted(folder_list) #오름 차순으로 정리 
for i,ele in enumerate(folder_list):
    try:
        print str(i)+'번째 data입니다.'
        subFolder_path=folder_path+ele
        subFolder_list=os.walk(subFolder_path).next()[1]
        print subFolder_list 
        if 'ROI_MASK' in subFolder_list:
            mask_list=os.walk(subFolder_path+'/ROI_MASK').next()[2] #...CCM_Data/Malignant/001'
            mask_path=attach_(subFolder_path+'/ROI_MASK'+'/',mask_list)
            mask_number=get_number_from_list(mask_list) #roimask_0002-->02
            mask_dic=make_dict(mask_number,mask_path)

            ct_list=os.walk(subFolder_path+'/'+subFolder_list[1]+'/').next()[2]
            ct_path=attach_(subFolder_path+'/'+subFolder_list[1]+'/',ct_list)
            ct_number=get_number_from_list(ct_list) #roimask_0002-->02
            ct_dic=make_dict(ct_number,ct_path)

            #print ct_dic.keys(); print ct_dic.values()
            cancer_mask_dic=find_mask(mask_dic)
            cancer_mask_locate=mask_locate(cancer_mask_dic)
            # cancer_mask_locate :top,buttom,left,right,center
            save_path = save_foler+'/'+ele
            if os.path.isdir(save_path)==True:
                continue;
            crop_mask_target_images_save(cancer_mask_dic, cancer_mask_locate , ct_dic ,save_path)
    except Exception as e:
        print str(e);
        continue;
        
        """
        에러가 뜨는 이유는 확실히다 처음 암 영역이 나온 이후 다시 
        나오기 때문이다. 그 알고리즈을 넣어야 한다 .
        
        """

        
        
        
        

0번째 data입니다.
['ROI_MASK', '3']
target_mask.keys():[12]
top , buttom (320, 61) (333, 66)
left , right (324, 58) (325, 71)
이미지를 crop 하고 저장합니다 
key 12
target_key 시작 위치 :701
target_key 끝 위치 :716
target_key:705
target_key_up:713
target_path /home/seongjung/ASAN_project1/project1/CCM_Data/Benign/001/3/1.3.12.2.1107.5.1.4.51914.30000006121806122618700000705.dcm
1번째 data입니다.
['ROI_MASK', '3']
target_mask.keys():[10]
top , buttom (308, 209) (325, 214)
left , right (315, 206) (316, 218)
이미지를 crop 하고 저장합니다 
key 10
target_key 시작 위치 :267
target_key 끝 위치 :286
target_key:277
target_key_up:277
target_path /home/seongjung/ASAN_project1/project1/CCM_Data/Benign/002/3/1.3.12.2.1107.5.1.4.51914.30000006022707104076500022277.dcm
2번째 data입니다.
['ROI_MASK', '3']
target_mask.keys():[19]
error info 
mask :  19
mask location: /home/seongjung/ASAN_project1/project1/CCM_Data/Benign/003/ROI_MASK/roimask_0019.dcm
이미지를 crop 하고 저장합니다 
key 19
target_key 시작 위치 :299
target_key 끝 위치 :334
target_key:316
target_key_up:318
t

In [332]:
>>> from operator import add
>>> map(add, list1, list2)
[5, 7, 9]

NameError: name 'list1' is not defined

In [ ]:
def change_all_name(list_):
    ret_list=[]
    for i,file_name in enumerate(list_):
        ret_list.append(change_file_name(file_name))
    
    return ret_list
def change_file_name(file_name):
    """
    1.3.12.2.1107.5.1.4.51914.30000006121806122618700000702.dcm
    -->30000006121806122618700000702.dcm
    -->name[-2:]=02
    """
    extension=file_name.split('.')[-1]
    name=file_name.split('.')[-2]
    name=name[-2:]
    """
    """
    return name
def make_dict(list_name , list_path):
    ret_dic={}
    for i,ele in enumerate(list_path):
        ret_dic[list_name[i]]=ele
    return ret_dic

In [ ]:
mask_path=crawl_folder('/home/seongjung/ASAN_project1/project1/CCM_Data/Benign/002/ROI_MASK')
mask_name=change_all_name(mask_path)
mask_name_int= map(int, mask_name)
#list__.sort(key=int)
print mask_name_int
mask_dic=make_dict(mask_name_int,mask_path)